In [121]:
%config IPCompleter.greedy=True
from operator import itemgetter
from time import time
import json


In [ ]:
#remove rare tags

    


In [362]:
#remove rare tags

with open('tag_prediction_data/pure_tags.json', 'r') as inf:
    tag_list = json.load(inf)

def remove_if_rare(tag):
    if tag not in tag_list:
        return 'no-tag'
    else:
        return tag

with open('tag_prediction_data/train_tags.msg', 'r') as inf:
    train_tags = list(map(str.strip, list(inf)))
    
with open('tag_prediction_data/train_tags_clean.msg', 'w') as ouf:
    for tag in train_tags:
        tag = remove_if_rare(tag)
        ouf.write(tag + '\n')
        
with open('tag_prediction_data/test_pure_tags.msg', 'r') as inf:
    test_tags = list(map(str.strip, list(inf)))
    
with open('tag_prediction_data/test_tags_clean.msg', 'w') as ouf:
    for tag in test_tags:
        tag = remove_if_rare(tag)
        ouf.write(tag + '\n')

In [363]:
with open('tag_prediction_data/train_fo.diff', 'r') as inf:
    train_diffs = list(map(str.strip, list(inf)))
    
with open('tag_prediction_data/train_tags_clean.msg', 'r') as inf:
    train_tags = list(map(str.strip, list(inf)))

with open('tag_prediction_data/test_fo.diff', 'r') as inf:
    test_diffs = list(map(str.strip, list(inf)))
    
with open('tag_prediction_data/test_tags_clean.msg', 'r') as inf:
    test_tags = list(map(str.strip, list(inf)))
    

In [364]:
#find best tag for all files from train set

files = {}

for i, diff in enumerate(train_diffs):
    cur_files = diff.split(' <nl> ')
    cur_tag = train_tags[i]
    for cur_file in cur_files:
        if cur_file not in files:
            files[cur_file] = {}
        if cur_tag not in files[cur_file]:
            files[cur_file][cur_tag] = 0
        files[cur_file][cur_tag] += 1

for file in files:
    tags = sorted(files[file].items(), key=itemgetter(1), reverse=True)
    best_tag = tags[0][0]
    files[file] = best_tag

In [365]:
splitted_files = []
for file in files:
    splitted_files.append(file.split())

In [366]:
def count_similarity(target, test):
    for i in range(min(len(target), len(test))):
        if target[i] != test[i]:
            return (i / len(target)) ** 2
    return min(1, len(test) / len(target))

def find_most_similar(test_file):
    best_file = []
    best_sim = -1
    for file in splitted_files:
        sim = count_similarity(file, test_file)
        if sim > best_sim:
            best_sim = sim
            best_file = file
    return files[' '.join(best_file)], best_sim

In [367]:
print(len(files))
len(test_diffs)

72209


12601

In [387]:
%%time
test_tags = []
similarities = {}
sim_threshold = 0.75

for i, diff in enumerate(test_diffs):
    t1 = time()
    cur_files = diff.split(' <nl> ')
    tags = {}
    
    for cur_file in cur_files:
        if cur_file in similarities:
            tag, sim = similarities[cur_file]
        else:
            tag, sim = find_most_similar(cur_file.split())
            similarities[cur_file] = (tag, sim)
        if tag not in tags:
            tags[tag] = sim
        else:
            tags[tag] = max(sim, tags[tag])
    t2 = time()
    if 'no-tag' in tags:
        tags['no-tag'] -= 0.01
    tags = sorted(tags.items(), key=itemgetter(1), reverse=True)
    print(i + 1, tags)
    if len(tags) == 0:
        test_tags.append('no-tag')
    elif len(tags) > 1 and tags[0][0] == 'no-tag':
        if tags[1][1] > sim_threshold:
            test_tags.append(tags[1][0])
        else:
            test_tags.append('no-tag')
    else:
        if tags[0][1] > sim_threshold:
            test_tags.append(tags[0][0])
        else:
            test_tags.append('no-tag')
    
    #print(t2 - t1)
    if (i % 20 == 0):
        print(i)

1 [('no-tag', 0.99)]
0
2 [('groovy', 1)]
3 [('groovy', 1), ('no-tag', 0.99)]
4 [('no-tag', 0.755625)]
5 [('no-tag', 0.99)]
6 [('no-tag', 0.8164462809917354)]
7 [('ea', 1), ('no-tag', 0.99)]
8 [('no-tag', 0.7801234567901234)]
9 [('no-tag', 0.7801234567901234)]
10 [('gui-test', 1), ('no-tag', 0.99)]
11 [('no-tag', 0.99)]
12 [('no-tag', 0.7801234567901234)]
13 [('no-tag', 0.99)]
14 [('no-tag', 0.99), ('gui-test', 0.765625)]
15 [('no-tag', 0.99)]
16 [('no-tag', 0.99)]
17 [('no-tag', 0.99)]
18 [('ssr', 0.7901234567901234)]
19 [('ssr', 1), ('no-tag', 0.99)]
20 [('ssr', 0.7901234567901234)]
21 [('ssr', 0.7901234567901234)]
20
22 [('ssr', 0.7901234567901234)]
23 [('ssr', 0.7901234567901234)]
24 [('file-history', 1)]
25 [('vcs-log', 0.81)]
26 [('no-tag', 0.755625)]
27 [('groovy', 1)]
28 [('no-tag', 0.99)]
29 [('no-tag', 0.99)]
30 [('no-tag', 0.755625)]
31 [('no-tag', 0.7801234567901234)]
32 [('no-tag', 0.99)]
33 [('no-tag', 0.5525)]
34 [('no-tag', 0.99)]
35 [('no-tag', 0.99)]
36 [('no-tag', 0.9

307 [('ruby', 1), ('no-tag', 0.99)]
308 [('no-tag', 0.99)]
309 [('no-tag', 0.99)]
310 [('no-tag', 0.99)]
311 [('no-tag', 0.99)]
312 [('no-tag', 0.99)]
313 [('no-tag', 0.99)]
314 [('no-tag', 0.6844444444444445)]
315 [('no-tag', 0.6844444444444445)]
316 [('no-tag', 0.99)]
317 [('no-tag', 0.99)]
318 [('no-tag', 0.7801234567901234)]
319 [('no-tag', 0.99), ('groovy', 0.81)]
320 [('no-tag', 0.99)]
321 [('no-tag', 0.99)]
320
322 [('no-tag', 0.99)]
323 [('no-tag', 0.99)]
324 [('no-tag', 0.99)]
325 [('no-tag', 0.7801234567901234)]
326 [('no-tag', 0.99)]
327 [('no-tag', 0.99)]
328 [('no-tag', 0.99)]
329 [('no-tag', 0.99)]
330 [('no-tag', 0.99)]
331 [('followup', 1), ('no-tag', 0.99)]
332 [('no-tag', 0.7801234567901234)]
333 [('no-tag', 0.99)]
334 [('ssr', 1)]
335 [('ig', 1), ('no-tag', 0.99)]
336 [('github', 1)]
337 [('github', 1), ('no-tag', 0.7801234567901234)]
338 [('no-tag', 0.7801234567901234)]
339 [('shelf', 1), ('lst', 1), ('no-tag', 0.99)]
340 [('vcs', 1)]
341 [('shelf', 1)]
340
342 [('n

611 [('no-tag', 0.6844444444444445)]
612 [('no-tag', 0.99)]
613 [('no-tag', -0.01)]
614 [('no-tag', 0.5525)]
615 [('no-tag', 0.99)]
616 [('no-tag', 0.99)]
617 [('no-tag', 0.99)]
618 [('no-tag', 0.99)]
619 [('no-tag', 0.99)]
620 [('no-tag', 0.99)]
621 [('no-tag', 0.99)]
620
622 [('no-tag', 0.99)]
623 [('vcs', 1), ('no-tag', 0.99)]
624 [('no-tag', 0.99)]
625 [('vcs-log', 1)]
626 [('no-tag', 0.99)]
627 [('no-tag', 0.99)]
628 [('no-tag', 0.99)]
629 [('no-tag', 0.99), ('ea', 0.765625)]
630 [('no-tag', 0.99)]
631 [('no-tag', 0.99)]
632 [('no-tag', 0.99)]
633 [('no-tag', 0.99)]
634 [('no-tag', 0.99)]
635 [('no-tag', 0.99)]
636 [('no-tag', 0.99)]
637 [('no-tag', 0.99)]
638 [('no-tag', 0.99)]
639 [('no-tag', 0.99)]
640 [('vcs-log', 1)]
641 [('no-tag', 0.99)]
640
642 [('no-tag', 0.99)]
643 [('no-tag', 0.99)]
644 [('no-tag', 0.755625)]
645 [('no-tag', 0.99)]
646 [('no-tag', 0.99), ('ea', 0.765625)]
647 [('no-tag', 0.99), ('followup', 0.765625)]
648 [('no-tag', 0.99)]
649 [('no-tag', 0.99)]
650 [(

919 [('vcs-log', 1)]
920 [('no-tag', 0.99)]
921 [('no-tag', 0.99)]
920
922 [('no-tag', 0.99)]
923 [('no-tag', 0.99)]
924 [('no-tag', 0.99)]
925 [('no-tag', 0.99)]
926 [('groovy', 1), ('no-tag', 0.99)]
927 [('no-tag', 0.99), ('platform', 0.7346938775510203)]
928 [('no-tag', 0.99)]
929 [('no-tag', 0.99)]
930 [('no-tag', 0.8)]
931 [('github', 1)]
932 [('no-tag', 0.7246938775510203)]
933 [('no-tag', 0.7246938775510203)]
934 [('no-tag', 0.99)]
935 [('github', 1), ('no-tag', 0.99)]
936 [('no-tag', 0.99)]
937 [('no-tag', 0.99)]
938 [('no-tag', 0.7246938775510203)]
939 [('no-tag', 0.99)]
940 [('no-tag', 0.99)]
941 [('no-tag', 0.99)]
940
942 [('no-tag', 0.8)]
943 [('no-tag', 0.99)]
944 [('no-tag', 0.99)]
945 [('no-tag', 0.99)]
946 [('no-tag', 0.99)]
947 [('no-tag', 0.99)]
948 [('no-tag', 0.99)]
949 [('no-tag', 0.99)]
950 [('no-tag', 0.99), ('gradle', 0.6944444444444445)]
951 [('no-tag', 0.24)]
952 [('no-tag', 0.99)]
953 [('no-tag', 0.99)]
954 [('no-tag', 0.99)]
955 [('no-tag', 0.99)]
956 [('no-

1215 [('no-tag', 0.99)]
1216 [('no-tag', 0.99)]
1217 [('no-tag', 0.99)]
1218 [('file-history', 0.81)]
1219 [('no-tag', 0.99), ('file-history', 0.81)]
1220 [('no-tag', 0.99)]
1221 [('no-tag', 0.99)]
1220
1222 [('groovy', 1)]
1223 [('no-tag', 0.8164462809917354)]
1224 [('no-tag', 0.99)]
1225 [('no-tag', 0.99)]
1226 [('no-tag', 0.99)]
1227 [('no-tag', 0.99)]
1228 [('no-tag', 0.755625)]
1229 [('no-tag', 0.99)]
1230 [('no-tag', 0.99)]
1231 [('no-tag', 0.99)]
1232 [('no-tag', 0.99)]
1233 [('no-tag', 0.99)]
1234 [('no-tag', 0.99)]
1235 [('no-tag', 0.8)]
1236 [('no-tag', 0.99)]
1237 [('vcs-log', 1)]
1238 [('no-tag', 0.99)]
1239 [('no-tag', 0.99)]
1240 [('no-tag', 0.99)]
1241 [('no-tag', 0.99)]
1240
1242 [('no-tag', 0.99)]
1243 [('no-tag', 0.99)]
1244 [('no-tag', 0.99), ('lst', 0.7346938775510203)]
1245 [('no-tag', 0.99)]
1246 [('no-tag', 0.99)]
1247 [('vcs-log', 1)]
1248 [('vcs-log', 1), ('no-tag', 0.99)]
1249 [('no-tag', 0.755625)]
1250 [('no-tag', 0.755625)]
1251 [('no-tag', 0.99)]
1252 [('n

...

11199 [('vcs', 1), ('vcs-log', 1), ('no-tag', 0.99)]
11200 [('no-tag', 0.99)]
11201 [('vcs', 1), ('no-tag', 0.99)]
11200
11202 [('no-tag', 0.99)]
11203 [('no-tag', 0.99)]
11204 [('no-tag', 0.99)]
11205 [('platform', 1), ('no-tag', 0.99)]
11206 [('no-tag', 0.99)]
11207 [('no-tag', 0.99)]
11208 [('no-tag', 0.99)]
11209 [('no-tag', 0.99)]
11210 [('no-tag', 0.99)]
11211 [('no-tag', 0.99)]
11212 [('no-tag', 0.8)]
11213 [('no-tag', 0.99)]
11214 [('no-tag', 0.8)]
11215 [('no-tag', 0.8)]
11216 [('no-tag', 0.99)]
11217 [('no-tag', 0.99)]
11218 [('no-tag', 0.99)]
11219 [('no-tag', 0.99)]
11220 [('no-tag', 0.755625)]
11221 [('no-tag', 0.99)]
11220
11222 [('no-tag', 0.99)]
11223 [('no-tag', 0.99)]
11224 [('no-tag', 0.99)]
11225 [('no-tag', 0.8164462809917354)]
11226 [('no-tag', 0.755625)]
11227 [('no-tag', 0.99)]
11228 [('ssr', 1)]
11229 [('no-tag', 0.99)]
11230 [('no-tag', 0.99)]
11231 [('no-tag', 0.99)]
11232 [('no-tag', 0.99)]
11233 [('no-tag', 0.99)]
11234 [('groovy', 1), ('no-tag', 0.99)]
112

11490 [('no-tag', 0.99)]
11491 [('no-tag', 0.99)]
11492 [('no-tag', 0.99)]
11493 [('no-tag', 0.99)]
11494 [('no-tag', 0.7246938775510203)]
11495 [('no-tag', 0.8)]
11496 [('no-tag', 0.99)]
11497 [('no-tag', 0.99)]
11498 [('no-tag', 0.99)]
11499 [('no-tag', 0.755625)]
11500 [('no-tag', 0.99)]
11501 [('no-tag', 0.99)]
11500
11502 [('no-tag', 0.99)]
11503 [('no-tag', 0.99)]
11504 [('no-tag', 0.99)]
11505 [('no-tag', 0.99)]
11506 [('git', 1), ('vcs-log', 1), ('no-tag', 0.99)]
11507 [('no-tag', 0.99)]
11508 [('no-tag', 0.99)]
11509 [('no-tag', 0.99)]
11510 [('no-tag', 0.99)]
11511 [('no-tag', 0.755625)]
11512 [('no-tag', 0.755625)]
11513 [('no-tag', 0.99)]
11514 [('no-tag', 0.99)]
11515 [('no-tag', 0.755625)]
11516 [('no-tag', 0.755625)]
11517 [('no-tag', 0.99)]
11518 [('no-tag', 0.99)]
11519 [('ig', 0.7901234567901234)]
11520 [('no-tag', 0.99)]
11521 [('no-tag', 0.99)]
11520
11522 [('no-tag', 0.99)]
11523 [('no-tag', 0.99)]
11524 [('no-tag', 0.99)]
11525 [('no-tag', 0.99)]
11526 [('no-tag',

11793 [('no-tag', 0.99)]
11794 [('no-tag', 0.99)]
11795 [('no-tag', 0.99)]
11796 [('no-tag', 0.99)]
11797 [('ssr', 1), ('no-tag', 0.99)]
11798 [('no-tag', 0.99), ('github', 0.6944444444444445)]
11799 [('no-tag', 0.99)]
11800 [('no-tag', 0.99)]
11801 [('no-tag', 0.99)]
11800
11802 [('no-tag', 0.99)]
11803 [('no-tag', 0.99)]
11804 [('no-tag', 0.99)]
11805 [('no-tag', 0.8)]
11806 [('no-tag', 0.8)]
11807 [('no-tag', 0.99)]
11808 [('no-tag', 0.99)]
11809 [('no-tag', 0.99)]
11810 [('no-tag', 0.99)]
11811 [('no-tag', 0.99)]
11812 [('no-tag', 0.99)]
11813 [('no-tag', 0.99)]
11814 [('no-tag', 0.99)]
11815 [('no-tag', 0.99)]
11816 [('no-tag', 0.99)]
11817 [('no-tag', 0.8)]
11818 [('no-tag', 0.99)]
11819 [('no-tag', 0.99)]
11820 [('no-tag', 0.99)]
11821 [('no-tag', 0.99)]
11820
11822 [('no-tag', 0.99)]
11823 [('no-tag', 0.99)]
11824 [('no-tag', 0.99)]
11825 [('no-tag', 0.99)]
11826 [('no-tag', 0.99)]
11827 [('no-tag', 0.99)]
11828 [('gradle', 1), ('no-tag', 0.99)]
11829 [('no-tag', 0.99)]
11830 [

12076 [('no-tag', 0.99)]
12077 [('no-tag', 0.99)]
12078 [('no-tag', 0.99)]
12079 [('no-tag', 0.99)]
12080 [('no-tag', 0.99)]
12081 [('no-tag', 0.99)]
12080
12082 [('no-tag', 0.99)]
12083 [('no-tag', 0.99)]
12084 [('no-tag', 0.99)]
12085 [('no-tag', 0.99)]
12086 [('no-tag', 0.99)]
12087 [('no-tag', 0.99)]
12088 [('vcs', 1), ('no-tag', 0.99)]
12089 [('no-tag', 0.7801234567901234)]
12090 [('no-tag', 0.99)]
12091 [('gradle', 0.6049382716049383)]
12092 [('no-tag', 0.99)]
12093 [('no-tag', 0.99)]
12094 [('no-tag', 0.99)]
12095 [('testng', 1), ('ea', 1), ('no-tag', 0.99)]
12096 [('no-tag', 0.99)]
12097 [('no-tag', 0.99)]
12098 [('vcs-log', 1), ('no-tag', 0.99)]
12099 [('vcs', 1), ('no-tag', 0.8)]
12100 [('no-tag', 0.99)]
12101 [('no-tag', 0.99)]
12100
12102 [('no-tag', 0.99)]
12103 [('no-tag', 0.99)]
12104 [('no-tag', 0.99)]
12105 [('vcs-log', 1), ('no-tag', 0.99)]
12106 [('vcs-log', 1)]
12107 [('vcs-log', 1)]
12108 [('vcs-log', 1), ('no-tag', 0.99)]
12109 [('vcs-log', 1)]
12110 [('diff', 1),

12377 [('no-tag', 0.99)]
12378 [('no-tag', 0.99)]
12379 [('no-tag', 0.99)]
12380 [('git', 1)]
12381 [('jvm-lang', 1), ('groovy', 1), ('no-tag', 0.99)]
12380
12382 [('no-tag', 0.99)]
12383 [('no-tag', 0.99)]
12384 [('no-tag', 0.99)]
12385 [('no-tag', 0.99)]
12386 [('no-tag', 0.99)]
12387 [('no-tag', 0.99)]
12388 [('no-tag', 0.99)]
12389 [('no-tag', 0.99)]
12390 [('no-tag', 0.99)]
12391 [('no-tag', 0.99)]
12392 [('no-tag', 0.99)]
12393 [('no-tag', 0.99)]
12394 [('no-tag', 0.99)]
12395 [('vcs', 1), ('no-tag', 0.99)]
12396 [('no-tag', 0.99)]
12397 [('no-tag', 0.99)]
12398 [('no-tag', 0.99)]
12399 [('no-tag', 0.99)]
12400 [('no-tag', 0.99)]
12401 [('no-tag', 0.99)]
12400
12402 [('no-tag', 0.99)]
12403 [('no-tag', 0.99)]
12404 [('no-tag', 0.99)]
12405 [('no-tag', 0.99), ('updater', 0.765625)]
12406 [('no-tag', 0.99)]
12407 [('no-tag', 0.99)]
12408 [('no-tag', 0.99)]
12409 [('no-tag', 0.99)]
12410 [('no-tag', 0.99)]
12411 [('no-tag', 0.99)]
12412 [('no-tag', 0.99)]
12413 [('no-tag', 0.99)]
12

In [388]:
with open('tag_prediction_data/generated_tags.msg', 'w') as ouf:
    for tag in test_tags:
        ouf.write(tag + '\n')

In [389]:
len(test_tags)

12601

In [396]:
#evaluation
with open('tag_prediction_data/test_tags_clean.msg', 'r') as test,\
     open('tag_prediction_data/generated_tags_3000_bestv.msg', 'r') as gen:
        test_tags = list(map(str.strip, list(test)))
        gen_tags = list(map(str.strip, list(gen)))
        n = 0
        cnt_exact_notag = 0
        cnt_exact = 0
        cnt_miss = 0
        cnt_notag = 0
        cnt_ambtag = 0
        for i in range(len(gen_tags)):
            if test_tags[i] == '':
                continue
            n += 1
            #gen_tags[i] = 'no-tag'
            if gen_tags[i] == test_tags[i]:
                if gen_tags[i] == 'no-tag':
                    cnt_exact_notag += 100
                else:
                    cnt_exact += 100
            elif gen_tags[i] == 'no-tag':
                cnt_notag += 100
            elif test_tags[i] == 'no-tag':
                cnt_ambtag += 100
#                 print(i + 1, gen_tags[i])
            else:
                cnt_miss += 100
        cnt_exact_notag /= n
        cnt_exact /= n
        cnt_miss /= n
        cnt_notag /= n
        cnt_ambtag /= n
        print(f'{cnt_exact_notag:.1f}% exactly predicted no-tags')
        cnt_exact += cnt_exact_notag
        print(f'{cnt_exact:.1f}% exactly predicted tags')
        print(f'{cnt_notag:.1f}% tags which wasn\'t predicted')
        print(f'{cnt_ambtag:.1f}% tags which was predicted but original has no tag')
        print(f'{cnt_miss:.1f}% tags which was predicted incorrect')

61.5% exactly predicted no-tags
71.2% exactly predicted tags
21.0% tags which wasn't predicted
4.8% tags which was predicted but original has no tag
3.0% tags which was predicted incorrect


In [337]:
gen_tags_set = set()
with open('tag_prediction_data/generated_tags_3000.msg', 'r') as inf:
    for line in inf:
        gen_tags_set.add(line.strip())
print(sorted(gen_tags_set))

test_tags_set = set()
with open('tag_prediction_data/test_tags_clean.msg', 'r') as inf:
    for line in inf:
        test_tags_set.add(line.strip())
print(sorted(test_tags_set))

train_tags_set = set()
with open('tag_prediction_data/train_tags_clean.msg', 'r') as inf:
    for line in inf:
        train_tags_set.add(line.strip())
print(sorted(train_tags_set))

tags_set = test_tags_set.union(train_tags_set)
tags_dict = {}
for tag in tags_set:
    tags_dict[tag] = ''
    
with open('pure_tags.json', 'w') as ouf:
    json.dump(dict(sorted(tags_dict.items())), ouf, indent=4)

['cleanup', 'configurecodestyleonselectedfragment', 'coverage', 'cpp', 'dbe', 'dea', 'deprecate', 'devkit', 'dfa', 'diff', 'ea', 'edu', 'emmet', 'file-history', 'followup', 'freeze logger', 'fus', 'gcutil', 'git', 'github', 'gpub', 'gradle', 'groovy', 'gui', 'gui-test', 'gui-tests', 'hg', 'idea', 'ig', 'inline', 'java', 'java decompiler', 'java formatter', 'javafx', 'jbzipfile', 'lst', 'lvti', 'merge', 'mvc', 'no-tag', 'oc', 'patch', 'platform', 'popups', 'project', 'properties', 'push', 'py', 'recent tests', 'refactor', 'regexp', 'rider', 'ruby', 'scopeview', 'shelf', 'ssr', 'stats-collector', 'terminal', 'text-completion', 'uast', 'ui', 'updater', 'ux', 'vcs', 'vcs-log', 'vfs', 'web', 'wi']
['', 'cleanup', 'coverage', 'devkit', 'dfa', 'diff', 'ea', 'editor', 'emmet', 'file-history', 'followup', 'fus', 'git', 'github', 'gradle', 'groovy', 'gui-test', 'hg', 'https', 'ig', 'inference', 'ipp', 'java', 'javafx', 'jetcheck', 'junit', 'jvm-lang', 'lst', 'maven', 'merge', 'minor', 'no-tag', 

In [349]:
files['community-guitests testSrc com intellij ide projectWizard kotlin createProject CreateGradleKotlinDslProjectWithKotlinGuiTest.kt']

'no-tag'

In [386]:
n

2038